<a href="https://colab.research.google.com/github/Rawan-Hasan/Predictive-Analysis-of-delhivery-Logistics_/blob/main/Delhivery_Feature_Engineering_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#                           DELHIVERY 🚚

---







---



In [86]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [18]:
df=pd.read_csv('/content/delhivery.csv')
df.head()

,data,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor
0,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:27:55,10.435660,14.0,11.0,11.9653,1.272727,14.0,11.0,11.9653,1.272727
1,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:17:55,18.936842,24.0,20.0,21.7243,1.200000,10.0,9.0,9.7590,1.111111
2,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:01:19.505586,27.637279,40.0,28.0,32.5395,1.428571,16.0,7.0,10.8152,2.285714
3,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 03:39:57,36.118028,62.0,40.0,45.5620,1.550000,21.0,12.0,13.0224,1.750000
4,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 03:33:55,39.386040,68.0,44.0,54.2181,1.545455,6.0,5.0,3.9153,1.200000


- Exploration of data

In [3]:
df.shape

(46413, 24)

In [4]:
df.columns

Index(['data', 'trip_creation_time', 'route_schedule_uuid', 'route_type',
       'trip_uuid', 'source_center', 'source_name', 'destination_center',
       'destination_name', 'od_start_time', 'od_end_time',
       'start_scan_to_end_scan', 'is_cutoff', 'cutoff_factor',
       'cutoff_timestamp', 'actual_distance_to_destination', 'actual_time',
       'osrm_time', 'osrm_distance', 'factor', 'segment_actual_time',
       'segment_osrm_time', 'segment_osrm_distance', 'segment_factor'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46413 entries, 0 to 46412
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   data                            46413 non-null  object 
 1   trip_creation_time              46413 non-null  object 
 2   route_schedule_uuid             46413 non-null  object 
 3   route_type                      46413 non-null  object 
 4   trip_uuid                       46413 non-null  object 
 5   source_center                   46413 non-null  object 
 6   source_name                     46292 non-null  object 
 7   destination_center              46413 non-null  object 
 8   destination_name                46337 non-null  object 
 9   od_start_time                   46413 non-null  object 
 10  od_end_time                     46413 non-null  object 
 11  start_scan_to_end_scan          46413 non-null  float64
 12  is_cutoff                       

- Duplicate Detection

In [6]:
df.duplicated().sum()

0

- Missing value

In [7]:
df.isnull().sum()/df.shape[0]*100

,0
data,0.000000
trip_creation_time,0.000000
route_schedule_uuid,0.000000
route_type,0.000000
trip_uuid,0.000000
source_center,0.000000
source_name,0.260703
destination_center,0.000000
destination_name,0.163747
od_start_time,0.000000


In [8]:
df.isna().any().T

,0
data,False
trip_creation_time,False
route_schedule_uuid,False
route_type,False
trip_uuid,False
source_center,False
source_name,True
destination_center,False
destination_name,True
od_start_time,False


- **Null Treatment:**

Handle null values in 'source_name' and 'destination_name' columns  through freq_imputer

In [19]:
freq_imputer = SimpleImputer(strategy = 'most_frequent')
for i in ['source_name', 'destination_name']:
    df[i] = pd.DataFrame(freq_imputer.fit_transform(pd.DataFrame(df[i])))


df.isna().any()

,0
data,False
trip_creation_time,False
route_schedule_uuid,False
route_type,False
trip_uuid,False
source_center,False
source_name,False
destination_center,False
destination_name,False
od_start_time,False


# Feature Engineering

Feature Engineering is the process of transforming raw data into meaningful features that better represent the underlying problem to the predictive models, resulting in improved model accuracy and performance. It is a critical step in the data science workflow that can greatly impact the outcome of a machine learning project.

- **Understand the Domain and Problem Statement**

Delhivery aims to establish itself as the premier player in the logistics industry. This case study is of paramount importance as it aligns with the company's core objectives and operational excellence.

It offers a practical framework for understanding and processing data, which is essential for their operations. By utilizing data engineering pipelines and data analysis techniques, Delhivery can accomplish several important goals.

- - **Objective**

 Logistics dataset focused on delivery time prediction with both actual and OSRM .




> Examine the Data



In [10]:
df.dtypes

,0
data,object
trip_creation_time,object
route_schedule_uuid,object
route_type,object
trip_uuid,object
source_center,object
source_name,object
destination_center,object
destination_name,object
od_start_time,object


In [20]:
# Converting the datatypes to category for columns
df['data']=df['data'].astype('category')
df['route_type']=df['route_type'].astype('category')

# Converting time columns to datetime format
for col in ['trip_creation_time', 'od_start_time', 'od_end_time']:
    df[col] = pd.to_datetime(df[col])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144867 entries, 0 to 144866
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   data                            144867 non-null  category      
 1   trip_creation_time              144867 non-null  datetime64[ns]
 2   route_schedule_uuid             144867 non-null  object        
 3   route_type                      144867 non-null  category      
 4   trip_uuid                       144867 non-null  object        
 5   source_center                   144867 non-null  object        
 6   source_name                     144867 non-null  object        
 7   destination_center              144867 non-null  object        
 8   destination_name                144867 non-null  object        
 9   od_start_time                   144867 non-null  datetime64[ns]
 10  od_end_time                     144867 non-null  datetim

In [22]:
# reducing the float64 to float32 to save memory
for col in df.select_dtypes(include='float64').columns:
    df[col]=df[col].astype('float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144867 entries, 0 to 144866
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   data                            144867 non-null  category      
 1   trip_creation_time              144867 non-null  datetime64[ns]
 2   route_schedule_uuid             144867 non-null  object        
 3   route_type                      144867 non-null  category      
 4   trip_uuid                       144867 non-null  object        
 5   source_center                   144867 non-null  object        
 6   source_name                     144867 non-null  object        
 7   destination_center              144867 non-null  object        
 8   destination_name                144867 non-null  object        
 9   od_start_time                   144867 non-null  datetime64[ns]
 10  od_end_time                     144867 non-null  datetim

 - **Feature Selection**
   - Task: Identify the most relevant features for the analysis or modeling task.

In [14]:
df.head()

,data,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor
0,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:27:55,10.435660,14.0,11.0,11.965300,1.272727,14.0,11.0,11.9653,1.272727
1,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:17:55,18.936842,24.0,20.0,21.724300,1.200000,10.0,9.0,9.7590,1.111111
2,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 04:01:19.505586,27.637280,40.0,28.0,32.539501,1.428571,16.0,7.0,10.8152,2.285714
3,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 03:39:57,36.118027,62.0,40.0,45.562000,1.550000,21.0,12.0,13.0224,1.750000
4,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,2018-09-20 03:33:55,39.386040,68.0,44.0,54.218102,1.545455,6.0,5.0,3.9153,1.200000




> Handle Multicollinearity





>>  Multicollinearity is a statistical phenomenon that occurs when two or more independent variables in a regression model are highly correlated, indicating a strong linear relationship among the predictor variables. This issue complicates regression analysis by making it difficult to accurately determine the individual effects of each independent variable on the dependent variable.







In [23]:
# Detecting Multicollinearity Using a Variance Inflation Factor (VIF)
def calculate_vif(df):
    vif = pd.DataFrame()
    vif['feature'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif
numerical_features = df.select_dtypes(include=[np.number])

# Calculate VIF
vif_df = calculate_vif(numerical_features)
vif_df

,feature,VIF
0,start_scan_to_end_scan,5.281888
1,cutoff_factor,54109.653305
2,actual_distance_to_destination,54591.195202
3,actual_time,50.222183
4,osrm_time,1028.005384
5,osrm_distance,1376.850323
6,factor,2.992717
7,segment_actual_time,3.186038
8,segment_osrm_time,33.215836
9,segment_osrm_distance,35.774723




*   VIF starts at 1 and has no upper limit

*   VIF = 1, no correlation between the independent variable and the other variables

**Features with very high VIF values, indicating significant multicollinearity**

- cutoff_factor: 54109.652471

- actual_distance_to_destination: 54591.194134

- osrm_time: 1028.005381

- osrm_distance: 1376.850323


In [24]:
# Remove features with the highest VIF
features_to_remove = ['cutoff_factor']

# Drop these features
data_remove = numerical_features.drop(columns=features_to_remove)

# Recalculate VIF
vif_remove_df = calculate_vif(data_remove)
vif_remove_df

,feature,VIF
0,start_scan_to_end_scan,5.248180
1,actual_distance_to_destination,269.040730
2,actual_time,49.535693
3,osrm_time,1026.345325
4,osrm_distance,1376.614915
5,factor,2.950124
6,segment_actual_time,3.178665
7,segment_osrm_time,33.175741
8,segment_osrm_distance,35.348261
9,segment_factor,2.004196


After removing cutoff_factor  the VIF values for actual_distance_to_destination is reduced.



>  Feature Importance Technique





- mutual information



The mutual information (MI) between two quantities is a measure of the extent to which knowledge of one quantity reduces uncertainty about the other.

In [25]:
lable = df['actual_time']
features = df.drop(columns=['actual_time'])
LE = LabelEncoder()

for col in features.columns:
    if features[col].dtype == 'object':
        features[col] = pd.to_numeric(features[col], errors='coerce')
        LE.fit(features[col].unique())
        features[col] = LE.transform(features[col])


In [ ]:
# mutual_info_regression with the encoded features
mi_scores = mutual_info_regression(features, lable)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=features.columns)
mi_scores = mi_scores.sort_values(ascending=False)
# Print  MI scores
print(mi_scores)

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

plt.figure(figsize=(8, 5))
plot_mi_scores(mi_scores)

The high-scoring Factor feature exhibits a strong relationship with the target.

- Random Forest feature importance

In [ ]:
RF=RandomForestRegressor()
RF.fit(features, lable)

rf_importances = pd.Series(rf.feature_importances_, index=features.columns)
rf_importances = rf_importances.sort_values(ascending=False)
# Print rf_importances
print(rf_importances)



> Remove Irrelevant Features



In [31]:
# Dropping
df=df.drop(['is_cutoff', 'cutoff_factor', 'cutoff_timestamp', 'factor', 'segment_factor'],axis=1)

**Create New Features**



> Mathematical Transformations



In [32]:
# calculating
df['od_duration_dirr_hour']=(df['od_end_time']-df['od_start_time']).dt.total_seconds()/3600

In [33]:
# Create ratio features
df['osrm_distance_time_ratio']=df['osrm_distance']/df['osrm_time']
df['distance_time_ratio']=df['actual_distance_to_destination']-df['actual_time']


In [34]:
# Time period of data
print(f"Max od_start_time: {df['od_start_time'].max()}\nMin od_start_time: {df['od_start_time'].min()}")
print(f"Range of od_start_time: {df['od_start_time'].max()-df['od_start_time'].min()}")

Max od_start_time: 2018-10-06 04:27:23.392375
Min od_start_time: 2018-09-12 00:00:16.535741
Range of od_start_time: 24 days 04:27:06.856634


In [35]:
# Time period of data
print(f"Max od_end_time: {df['od_end_time'].max()}\nMin od_end_time: {df['od_end_time'].min()}")
print(f"Range of od_end_time: {df['od_end_time'].max()-df['od_end_time'].min()}")

Max od_end_time: 2018-10-08 03:00:24.353479
Min od_end_time: 2018-09-12 00:50:10.814399
Range of od_end_time: 26 days 02:10:13.539080




> Aggregations



In [36]:
# Merge trip_uuid and route_schedule_uuid
df['trip_route_id'] = df['trip_uuid'] + '_' + df['route_schedule_uuid']

In [37]:
# Aggregate by segment
segment_columns = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']
df['segment_key'] = df['trip_uuid'] + '+' + df['source_center'] + '+' + df['destination_center']

for col in segment_columns:
    df[col + '_sum'] = df.groupby('segment_key')[col].cumsum()

df[['segment_key', 'segment_actual_time', 'segment_actual_time_sum','segment_osrm_distance', 'segment_osrm_distance_sum','segment_osrm_time', 'segment_osrm_time_sum']]

,segment_key,segment_actual_time,segment_actual_time_sum,segment_osrm_distance,segment_osrm_distance_sum,segment_osrm_time,segment_osrm_time_sum
0,trip-153741093647649320+IND388121AAA+IND388620AAB,14.0,14.0,11.965300,11.965300,11.0,11.0
1,trip-153741093647649320+IND388121AAA+IND388620AAB,10.0,24.0,9.759000,21.724300,9.0,20.0
2,trip-153741093647649320+IND388121AAA+IND388620AAB,16.0,40.0,10.815200,32.539497,7.0,27.0
3,trip-153741093647649320+IND388121AAA+IND388620AAB,21.0,61.0,13.022400,45.561897,12.0,39.0
4,trip-153741093647649320+IND388121AAA+IND388620AAB,6.0,67.0,3.915300,49.477200,5.0,44.0
...,...,...,...,...,...,...,...
144862,trip-153746066843555182+IND131028AAB+IND000000ACB,12.0,92.0,8.185800,65.348701,12.0,94.0
144863,trip-153746066843555182+IND131028AAB+IND000000ACB,26.0,118.0,17.372499,82.721199,21.0,115.0
144864,trip-153746066843555182+IND131028AAB+IND000000ACB,20.0,138.0,20.705299,103.426498,34.0,149.0
144865,trip-153746066843555182+IND131028AAB+IND000000ACB,17.0,155.0,18.888500,122.315002,27.0,176.0




> Date and Time Features



In [38]:
# Extracted hour, day name, and month name from 'trip_creation_time'
df['hour_of_ day']=df['trip_creation_time'].dt.hour
df['day_of_week']=df['trip_creation_time'].dt.day_name()
df['month_of_year']=df['trip_creation_time'].dt.month_name()

In [48]:
# Extract date and time-based features
if 'cutoff_timestamp' in df.columns:
    df['cutoff_hour'] = df['cutoff_timestamp'].dt.hour
    df['cutoff_day'] = df['cutoff_timestamp'].dt.day
    df['cutoff_month'] = df['cutoff_timestamp'].dt.month
    df['cutoff_year'] = df['cutoff_timestamp'].dt.year

In [49]:
# Extract city ,code and state from destination_name
df[['destination','destination_state']] = df['destination_name'].str.split('(', n=1, expand=True)
df['destination_state'] = df['destination_state'].str.rstrip(')')
df[['destination_City','destination_place','destination_code']] = df['destination'].str.split('_', n=2, expand=True)


# Extract city ,code and state from source_name
df[['source','source_state']] = df['source_name'].str.split('(', n=1, expand=True)
df['source_state'] = df['source_state'].str.rstrip(')')
df[['source_City','source_place','source_code']] = df['source'].str.split('_', n=2, expand=True)

# Verify the updated DataFrame
df.head()

,data,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,destination,destination_state,destination_City,destination_place,destination_code,source,source_state,source_City,source_place,source_code
0,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,Khambhat_MotvdDPP_D,Gujarat,Khambhat,MotvdDPP,D,Anand_VUNagar_DC,Gujarat,Anand,VUNagar,DC
1,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,Khambhat_MotvdDPP_D,Gujarat,Khambhat,MotvdDPP,D,Anand_VUNagar_DC,Gujarat,Anand,VUNagar,DC
2,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,Khambhat_MotvdDPP_D,Gujarat,Khambhat,MotvdDPP,D,Anand_VUNagar_DC,Gujarat,Anand,VUNagar,DC
3,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,Khambhat_MotvdDPP_D,Gujarat,Khambhat,MotvdDPP,D,Anand_VUNagar_DC,Gujarat,Anand,VUNagar,DC
4,training,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,Khambhat_MotvdDPP_D,Gujarat,Khambhat,MotvdDPP,D,Anand_VUNagar_DC,Gujarat,Anand,VUNagar,DC




> Text Features



 - TF-IDF

 It can be defined as the calculation of how relevant a word in a series or corpus is to a text. The meaning increases proportionally to the number of times in the text a word appears but is compensated by the word frequency in the corpus (data-set).

In [51]:
# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

text_data = df['destination_name'].dropna().values

# Now you can use text_data
tfidf_features = tfidf_vectorizer.fit_transform(text_data)

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

- **Feature Transformation**



> Normalization



In [53]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
min_max_scaler = MinMaxScaler()

# Fit and transform the data (example for numerical features)
min_max_scaled_numerical = min_max_scaler.fit_transform(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
min_max_scaled_df = pd.DataFrame(min_max_scaled_numerical, columns=df.select_dtypes(include=[np.number]).columns)
min_max_scaled_df

,start_scan_to_end_scan,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time,segment_osrm_time,segment_osrm_distance,od_duration_dirr_hour,osrm_distance_time_ratio,distance_time_ratio,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,hour_of_ day
0,0.008378,0.000748,0.001105,0.002976,0.001276,0.078300,0.006828,0.005460,0.008316,0.514683,0.947065,0.001112,0.001124,0.002588,0.086957
1,0.008378,0.005180,0.003316,0.008333,0.005488,0.077086,0.005587,0.004453,0.008316,0.513501,0.946567,0.003337,0.004831,0.007246,0.086957
2,0.008378,0.009715,0.006854,0.013095,0.010155,0.078907,0.004345,0.004935,0.008316,0.571778,0.944140,0.006897,0.008941,0.010870,0.086957
3,0.008378,0.014135,0.011718,0.020238,0.015775,0.080425,0.007449,0.005942,0.008316,0.554063,0.939644,0.011568,0.013889,0.017081,0.086957
4,0.008378,0.015839,0.013044,0.022619,0.019511,0.075873,0.003104,0.001787,0.008316,0.625598,0.938735,0.012903,0.015376,0.019669,0.086957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144862,0.051663,0.018900,0.018793,0.032143,0.025427,0.077693,0.007449,0.003735,0.051662,0.548753,0.932042,0.018465,0.021407,0.045549,0.695652
144863,0.051663,0.023505,0.024541,0.041667,0.033090,0.081942,0.013035,0.007928,0.051662,0.545124,0.926334,0.024249,0.028007,0.056418,0.695652
144864,0.051663,0.029797,0.028963,0.048810,0.038014,0.080121,0.021105,0.009448,0.051662,0.526642,0.923697,0.028699,0.035874,0.074017,0.695652
144865,0.051663,0.033715,0.032943,0.054762,0.044132,0.079211,0.016760,0.008619,0.051662,0.551274,0.920211,0.032481,0.043051,0.087992,0.695652




> Standardization



 Standardization works only with data which follows normal distribution

In [55]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
standard_scaler = StandardScaler()

# Fit and transform the data
standardized_data = standard_scaler.fit_transform(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
std_scaled_df = pd.DataFrame(standardized_data, columns=df.select_dtypes(include=[np.number]).columns)
std_scaled_df

,start_scan_to_end_scan,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time,segment_osrm_time,segment_osrm_distance,od_duration_dirr_hour,osrm_distance_time_ratio,distance_time_ratio,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,hour_of_ day
0,-0.844026,-0.648246,-0.673677,-0.658642,-0.647814,-0.414331,-0.508094,-0.608251,-0.844299,-0.845722,0.664159,-0.673626,-0.638674,-0.647992,-1.340026
1,-0.844026,-0.623604,-0.656958,-0.629422,-0.624640,-0.488998,-0.643449,-0.731779,-0.844299,-0.854063,0.658607,-0.656734,-0.618185,-0.622821,-1.340026
2,-0.844026,-0.598385,-0.630207,-0.603449,-0.598958,-0.376997,-0.778805,-0.672644,-0.844299,-0.442761,0.631567,-0.629706,-0.595480,-0.603244,-1.340026
3,-0.844026,-0.573802,-0.593424,-0.564489,-0.568034,-0.283663,-0.440416,-0.549064,-0.844299,-0.567788,0.581486,-0.594233,-0.568140,-0.569684,-1.340026
4,-0.844026,-0.564329,-0.583392,-0.551502,-0.547479,-0.563666,-0.914160,-1.058963,-0.844299,-0.062913,0.571366,-0.584098,-0.559920,-0.555701,-1.340026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144862,-0.515196,-0.547308,-0.539921,-0.499556,-0.514923,-0.451665,-0.440416,-0.819862,-0.515013,-0.605264,0.496805,-0.541868,-0.526599,-0.415866,0.421094
144863,-0.515196,-0.521700,-0.496450,-0.447610,-0.472762,-0.190329,0.168683,-0.305506,-0.515013,-0.630875,0.433216,-0.497949,-0.490127,-0.357136,0.421094
144864,-0.515196,-0.486711,-0.463011,-0.408650,-0.445666,-0.302330,1.048494,-0.118905,-0.515013,-0.761313,0.403844,-0.464165,-0.446658,-0.262048,0.421094
144865,-0.515196,-0.464921,-0.432916,-0.376183,-0.412000,-0.358330,0.574750,-0.220626,-0.515013,-0.587471,0.365011,-0.435449,-0.407003,-0.186537,0.421094




> Log Transformation:



In [56]:
# Apply log transformation
log_transformed_data = np.log1p(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
log_transformed_df = pd.DataFrame(log_transformed_data, columns=df.select_dtypes(include=[np.number]).columns)
log_transformed_df

,start_scan_to_end_scan,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time,segment_osrm_time,segment_osrm_distance,od_duration_dirr_hour,osrm_distance_time_ratio,distance_time_ratio,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,hour_of_ day
0,4.465908,2.436737,2.708050,2.484907,2.562277,2.708050,2.484907,2.562277,0.890724,0.736089,NaN,2.708050,2.562277,2.484907,1.098612
1,4.465908,2.992569,3.218876,3.044523,3.123435,2.397895,2.302585,2.375743,0.890724,0.735351,NaN,3.218876,3.123435,3.044523,1.098612
2,4.465908,3.354709,3.713572,3.367296,3.512724,2.833213,2.079442,2.469387,0.890724,0.771092,NaN,3.713572,3.512724,3.332205,1.098612
3,4.465908,3.614103,4.143135,3.713572,3.840785,3.091043,2.564949,2.640656,0.890724,0.760362,NaN,4.127134,3.840783,3.688879,1.098612
4,4.465908,3.698484,4.234107,3.806663,4.011291,1.945910,1.791759,1.592353,0.890724,0.803001,NaN,4.219508,3.921522,3.806663,1.098612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144862,6.059123,3.834240,4.553877,4.110874,4.233062,2.564949,2.564949,2.217659,2.095328,0.757123,NaN,4.532599,4.194924,4.553877,2.833213
144863,6.059123,4.009014,4.795791,4.343805,4.462256,3.295837,3.091043,2.910855,2.095328,0.754904,NaN,4.779123,4.427492,4.753590,2.833213
144864,6.059123,4.207131,4.948760,4.488636,4.585919,3.044523,3.555348,3.077556,2.095328,0.743523,NaN,4.934474,4.648483,5.010635,2.833213
144865,6.059123,4.313221,5.068904,4.595120,4.720915,2.890372,3.332205,2.990142,2.095328,0.758662,NaN,5.049856,4.814742,5.176150,2.833213




> Polynomial Features



In [57]:
from sklearn.preprocessing import PolynomialFeatures

# Initialize PolynomialFeatures (degree=2 for simplicity)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Fit and transform the data
poly_features = poly.fit_transform(df.select_dtypes(include=[np.number]))

# Get feature names
poly_feature_names = poly.get_feature_names_out(df.select_dtypes(include=[np.number]).columns)

# Convert to DataFrame
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)




>  Quantile Transformation



In [58]:
from sklearn.preprocessing import QuantileTransformer

# Initialize QuantileTransformer (output_distribution='normal' for Gaussian-like)
quantile_transformer = QuantileTransformer(output_distribution='normal')

# Fit and transform the data using select_dtypes to get numerical columns
quantile_transformed_data = quantile_transformer.fit_transform(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
quantile_df = pd.DataFrame(quantile_transformed_data, columns=df.select_dtypes(include=[np.number]).columns)
quantile_df

,start_scan_to_end_scan,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time,segment_osrm_time,segment_osrm_distance,od_duration_dirr_hour,osrm_distance_time_ratio,distance_time_ratio,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,hour_of_ day
0,-1.337127,-1.432643,-2.211127,-1.761948,-1.750162,-1.020888,-0.676854,-0.685069,-1.340343,-0.780962,1.974980,-2.211127,-1.750771,-1.761948,-1.044409
1,-1.337127,-1.033244,-1.529558,-1.020888,-1.145796,-1.394512,-0.879040,-1.047965,-1.340343,-0.788801,1.751648,-1.525529,-1.154190,-1.012474,-1.044409
2,-1.337127,-0.509446,-0.908975,-0.637923,-0.585536,-0.880888,-1.164444,-0.843318,-1.340343,-0.509023,1.152195,-0.903301,-0.606508,-0.678433,-1.044409
3,-1.337127,-0.362292,-0.509202,-0.317650,-0.348899,-0.596935,-0.590949,-0.575110,-1.340343,-0.573375,0.569543,-0.514923,-0.376273,-0.362850,-1.044409
4,-1.337127,-0.295981,-0.432104,-0.239380,-0.198285,-1.710968,-1.482760,-1.656989,-1.340343,-0.322875,0.501565,-0.434861,-0.303263,-0.274391,-1.044409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144862,-0.024270,-0.155666,-0.200818,-0.030114,-0.073655,-1.179402,-0.590949,-1.286774,-0.024269,-0.592140,0.152002,-0.209787,-0.109293,0.163824,0.094232
144863,-0.024270,-0.059520,-0.042668,0.103057,0.062018,-0.207223,0.529301,-0.383824,-0.024269,-0.608396,-0.028922,-0.047692,0.016198,0.265284,0.094232
144864,-0.024270,0.012531,0.046436,0.188033,0.120591,-0.659602,1.501861,-0.296725,-0.024269,-0.699166,-0.098893,0.042668,0.125828,0.387078,0.094232
144865,-0.024270,0.120509,0.111890,0.244548,0.202159,-0.821412,1.075207,-0.352644,-0.024269,-0.582088,-0.179118,0.108103,0.211855,0.469603,0.094232


- **Feature Scaling**



>  Min-Max Scaling



In [59]:
# Initialize the MinMaxScaler with range [-1, 1]
min_max_scaler_neg1_1 = MinMaxScaler(feature_range=(-1, 1))

# Fit and transform the data
min_max_scaled_data_neg1_1 = min_max_scaler_neg1_1.fit_transform(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
min_max_scaled_df_neg1_1 = pd.DataFrame(min_max_scaled_data_neg1_1, columns=df.select_dtypes(include=[np.number]).columns)




> Robust Scaling



In [61]:
from sklearn.preprocessing import RobustScaler

# Initialize the RobustScaler
robust_scaler = RobustScaler()

# Fit and transform the data
robust_scaled_data = robust_scaler.fit_transform(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
robust_scaled_df = pd.DataFrame(robust_scaled_data, columns=df.select_dtypes(include=[np.number]).columns)




> Max-Abs Scaling



In [62]:
from sklearn.preprocessing import MaxAbsScaler

# Initialize the MaxAbsScaler
max_abs_scaler = MaxAbsScaler()

# Fit and transform the data
max_abs_scaled_data = max_abs_scaler.fit_transform(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
max_abs_scaled_df = pd.DataFrame(max_abs_scaled_data, columns=df.select_dtypes(include=[np.number]).columns)


- **Feature Reduction**



> Principal Component Analysis (PCA)



In [65]:
from sklearn.decomposition import PCA

# Initialize PCA
pca = PCA(n_components=5)

# Fit and transform the data
pca_data = pca.fit_transform(df.select_dtypes(include=[np.number]))

# Convert to DataFrame
pca_df = pd.DataFrame(pca_data)
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

Explained Variance Ratio: [0.88760989 0.10057549 0.00895167 0.00132298 0.00095282]




> Feature Selection Techniques



- ANOVA

In [84]:
# Apply ANOVA
anova_selector = SelectKBest(f_classif, k=10)
X_anova_selected = anova_selector.fit_transform(X, y)

# Get the selected feature names
anova_selected_features = X.columns[anova_selector.get_support()]




>  Embedded Methods



- Lasso Regression

In [80]:
# Identify  non-numeric columns
numeric_cols = X.select_dtypes(include=['number']).columns
X_numeric = X[numeric_cols]
X_scaled = StandardScaler().fit_transform(X_numeric)

# Fit Lasso model with cross-validation
lasso = LassoCV(cv=5)
lasso.fit(X_scaled, y)

# Get the coefficients
lasso_coefficients = pd.Series(lasso.coef_, index=numeric_cols)



> Recursive Feature Elimination (RFE)



In [85]:
model = LinearRegression()
# Initialize RFE
rfe = RFE(model, n_features_to_select=10)
rfe.fit(X, y)
rfe_selected_features = X.columns[rfe.get_support()]